In [50]:
import pandas as pd

In [51]:
xl = pd.ExcelFile('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx')

xl.sheet_names  # see all sheet names

['percent GDP', 'GDP']

In [52]:
GDP = xl.parse("GDP")  # read a specific sheet to DataFrame

In [53]:
GDP2 = GDP.set_index(["wbregionname","year","ssp"]).gdp.unstack("wbregionname")

In [54]:
GDP2.columns

Index(['East Asia and Pacific', 'Europe and Central Asia',
       'Latin America and Caribbean', 'Middle East and North Africa',
       'South Asia', 'Sub-Saharan Africa'],
      dtype='object', name='wbregionname')

In [55]:
GDP2 = GDP2.rename(columns={"Europe and Central Asia":"Eastern Europe and Central Asia",
                            "Middle East and North Africa":"Middle-East and North Africa"})

In [56]:
GDP2["Africa"] = GDP2["Middle-East and North Africa"]+GDP2["Sub-Saharan Africa"]
GDP2["Asia"] = GDP2["East Asia and Pacific"]+GDP2["South Asia"]
GDP2["all_countries"] = GDP2["Africa"]+GDP2["Asia"]+GDP2["Latin America and Caribbean"]+GDP2["Eastern Europe and Central Asia"]

In [57]:
GDP2.stack().sample(5)

year  ssp   wbregionname                
2026  ssp3  Middle-East and North Africa    3.505649e+12
2023  ssp1  East Asia and Pacific           2.302950e+13
2025  ssp1  Africa                          7.758734e+12
      ssp5  all_countries                   5.918826e+13
2020  ssp5  East Asia and Pacific           1.901538e+13
dtype: float64

In [58]:
res = xl.parse("percent GDP")

In [59]:
res["sector"] = res.sector.replace({"WASH":"wash"})

In [60]:
res.region.unique()

array(['all_countries', 'Africa', 'Asia', 'Latin America and Caribbean',
       'Former soviet Union', 'Middle-East and North Africa',
       'Sub-Saharan Africa', 'South Asia', 'East Asia and Pacific',
       'Eastern Europe and Central Asia'], dtype=object)

In [61]:
res["region"] = res.region.replace({'Former soviet Union':'Eastern Europe and Central Asia'})

In [62]:
res.loc[res.sector=="energy","ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="transport")&(res.scenario=="max"),"ssp"] = "ssp3"
res.loc[(res.sector=="wash")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="max"),"ssp"] = "ssp4"
res.loc[(res.sector=="coastal protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="max"),"ssp"] = "ssp5"
res.loc[(res.sector=="river protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="river protection")&(res.scenario=="min"),"ssp"] = "ssp3"
res.loc[(res.sector=="river protection")&(res.scenario=="max"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="irrigation")&(res.scenario=="max"),"ssp"] = "ssp2"

res.loc[(res.region=="all_countries"),"ssp"] = "ssp2"

In [63]:
res[res.ssp.isnull()]

,region,sector,scenario,variable,capital vs maintenance,data,ssp


In [64]:
mydata = res.merge(GDP2.stack().reset_index().rename(columns={"wbregionname":"region",0:"GDP"}),on=["region","ssp"])

In [65]:
mydata["cost_bUSD"] = 1e-9/100*mydata.data*mydata.GDP

In [66]:
d = 0.06
mydata["discount_factor"] = (1+d)**(1-(mydata.year.astype(int)-2015))

In [67]:
mydata["cost_bUSD_disc"] = mydata["cost_bUSD"]*mydata["discount_factor"]

In [68]:
newdata = mydata.set_index(["region","sector","scenario","capital vs maintenance","year"]).cost_bUSD_disc.sum(level=["region","sector","scenario","capital vs maintenance"])

In [69]:
newdata2 = newdata.reset_index()

In [70]:
newdata2["cost_bUSD_disc_annual"] = newdata2["cost_bUSD_disc"]/15

In [71]:
totals = pd.DataFrame()

# Energy

In [72]:
energy = newdata2.loc[(newdata2.sector=="energy"),:]

## keep for overview

In [73]:
energy.loc[energy.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()

,scenario,capital vs maintenance,cost_bUSD_disc_annual
0,best,capital,778.114139
1,min,capital,298.587895
2,max,capital,1020.534268
3,best,maintenance,206.098170
4,min,maintenance,107.185252
5,max,maintenance,185.356516


In [74]:
temp = energy.loc[energy.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "energy"

In [75]:
totals = temp

In [76]:
energy.loc[energy.region=="all_countries",["scenario","cost_bUSD_disc_annual","capital vs maintenance"]]

,scenario,cost_bUSD_disc_annual,capital vs maintenance
0,min,105.895694,maintenance
5,best,211.791389,maintenance
10,max,282.388519,maintenance
15,min,317.687083,capital
20,best,776.568426,capital
25,max,1058.956944,capital


# Transport

In [77]:
transport = newdata2.loc[(newdata2.sector=="transport"),:]

## keep for overview

In [78]:
transport.loc[transport.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                    416.599217
          maintenance                460.998755
min       capital                    156.796355
          maintenance                552.470203
max       capital                   1060.207794
          maintenance                699.317720
Name: cost_bUSD_disc_annual, dtype: float64

In [79]:
temp = transport.loc[transport.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "transport"

In [80]:
totals = totals.append(temp)

In [81]:
transport.loc[transport.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
1,min,388.284213
6,best,458.881343
11,max,741.269861
16,min,187.082394
21,best,458.881343
26,max,1164.852639


# WASH

In [82]:
wash = newdata2.loc[(newdata2.sector=="wash"),:]

## keep for overview

In [83]:
wash.loc[wash.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                   198.148016
          maintenance                67.185574
min       capital                   115.630826
          maintenance                31.883513
max       capital                   228.925546
          maintenance                68.875119
Name: cost_bUSD_disc_annual, dtype: float64

In [84]:
temp = wash.loc[wash.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "wash"

In [85]:
totals = totals.append(temp)

In [86]:
wash.loc[wash.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
2,min,169.433111
7,best,264.739236
12,max,264.739236
17,min,112.955407
22,best,194.142106
27,max,229.440671


# Flood

In [87]:
coastal_flood = newdata2.loc[(newdata2.sector=="coastal protection"),:]

In [88]:
coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                   42.919258
          maintenance               21.333604
min       capital                    2.349903
          maintenance                6.152358
max       capital                   56.573056
          maintenance               23.854950
Name: cost_bUSD_disc_annual, dtype: float64

In [89]:
river_flood = newdata2.loc[(newdata2.sector=="river protection"),:]

In [90]:
river_flood.loc[river_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                    60.137460
          maintenance                 3.204862
max       capital                   278.575892
          maintenance                14.260740
min       capital                    20.589857
          maintenance                 1.073654
Name: cost_bUSD_disc_annual, dtype: float64

In [91]:
river_flood.loc[river_flood.region=="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
maxmax    capital                   847.165556
minmin    capital                    14.119426
Name: cost_bUSD_disc_annual, dtype: float64

In [92]:
flood = coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])+\
river_flood.loc[river_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

## keep for overview

In [93]:
flood

scenario  capital vs maintenance
best      capital                   103.056717
          maintenance                24.538467
max       capital                   335.148948
          maintenance                38.115690
min       capital                    22.939760
          maintenance                 7.226012
Name: cost_bUSD_disc_annual, dtype: float64

In [94]:
temp = flood.reset_index()
temp["sector"] = "flood"

In [95]:
totals = totals.append(temp)

In [96]:
newdata2.loc[(newdata2.sector=="flood"),:]

,region,sector,scenario,capital vs maintenance,cost_bUSD_disc,cost_bUSD_disc_annual
3,all_countries,flood,min,maintenance,74.126986,4.941799
8,all_countries,flood,best,maintenance,370.634931,24.708995
13,all_countries,flood,max,maintenance,741.269861,49.417991
18,all_countries,flood,min,capital,317.687083,21.179139
23,all_countries,flood,best,capital,1694.331111,112.955407
28,all_countries,flood,max,capital,5294.784722,352.985648


# Irrigation

In [97]:
irrigation = newdata2.loc[(newdata2.sector=="irrigation"),:]

## keep for overview

In [98]:
irrigation.loc[irrigation.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                    49.619364
max       capital                   100.434281
min       capital                    43.592279
Name: cost_bUSD_disc_annual, dtype: float64

In [99]:
temp = irrigation.loc[irrigation.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "irrigation"

In [100]:
totals = totals.append(temp)

In [101]:
irrigation.loc[irrigation.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
4,min,0.000000
9,best,0.000000
14,max,0.000000
19,min,42.358278
24,best,45.888134
29,max,70.597130


In [102]:
newdata2[(newdata2.region!="all_countries")].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                   1545.537453
min       capital                    637.547114
max       capital                   2745.250838
best      maintenance                758.820966
min       maintenance                698.764981
max       maintenance                991.665046
Name: cost_bUSD_disc_annual, dtype: float64

In [104]:
newdata2[(newdata2.region=="all_countries")&(newdata2["capital vs maintenance"]=="capital")].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
min        681.262301
best      1588.435417
max       2876.833032
maxmax     847.165556
minmin      14.119426
Name: cost_bUSD_disc_annual, dtype: float64

In [105]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance")

capital vs maintenance      capital  maintenance
sector     scenario                             
energy     best          778.114139   206.098170
           max          1020.534268   185.356516
           min           298.587895   107.185252
flood      best          103.056717    24.538467
           max           335.148948    38.115690
           min            22.939760     7.226012
irrigation best           49.619364          NaN
           max           100.434281          NaN
           min            43.592279          NaN
transport  best          416.599217   460.998755
           max          1060.207794   699.317720
           min           156.796355   552.470203
wash       best          198.148016    67.185574
           max           228.925546    68.875119
           min           115.630826    31.883513

In [106]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").sum(level="scenario")

capital vs maintenance,capital,maintenance
scenario,,
best,1545.537453,758.820966
max,2745.250838,991.665046
min,637.547114,698.764981


In [107]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_usd_results.csv')

In [108]:
res[res.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance"]).data.unstack("capital vs maintenance")

capital vs maintenance     capital  maintenance
sector           scenario                      
energy           best         2.20        0.600
                 max          3.00        0.800
                 min          0.90        0.300
flood            best         0.32        0.070
                 max          1.00        0.140
                 min          0.06        0.014
irrigation       best         0.13          NaN
                 max          0.20          NaN
                 min          0.12          NaN
river protection maxmax       2.40          NaN
                 minmin       0.04          NaN
transport        best         1.30        1.300
                 max          3.30        2.100
                 min          0.53        1.100
wash             best         0.55        0.750
                 max          0.65        0.750
                 min          0.32        0.480

In [109]:
res[res.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance"]).data.unstack("capital vs maintenance").to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_percent_results.csv')

In [111]:
region_totals = newdata2.set_index(["region","sector","scenario","capital vs maintenance"]).drop("all_countries",level="region").sum(level=["region","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").capital

In [112]:
region_totals = region_totals.reset_index()

In [113]:
region_totals['region2'] = region_totals.region

region_totals.loc[region_totals.region=='Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Middle-East and North Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Sub-Saharan Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Asia','region2']='Asia'
region_totals.loc[region_totals.region=='South Asia','region2']='Asia'
region_totals.loc[region_totals.region=='East Asia and Pacific','region2']='Asia'

In [114]:
region_totals2 = region_totals.set_index(['region2','scenario']).capital.sum(level=["region2",'scenario'])

In [115]:
region_totals2 = region_totals2.unstack("scenario")

In [116]:
region_totals2.sum(axis=0)

scenario
best    1545.537453
max     2745.250838
min      637.547114
dtype: float64

In [117]:
region_totalspc = region_totals2/region_totals2.sum(axis=0)

In [118]:
region_totalspc

scenario,best,max,min
region2,,,
Africa,0.204224,0.225473,0.228065
Asia,0.512672,0.504093,0.493327
Eastern Europe and Central Asia,0.118324,0.059585,0.054547
Latin America and Caribbean,0.164781,0.210849,0.224061


faux: ne pas faire le truc ci-dessous (a corriger)

In [162]:
region_pc_totals = mydata.copy()
region_pc_totals['region2'] = region_pc_totals.region

region_pc_totals.loc[region_pc_totals.region=='Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Middle-East and North Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Sub-Saharan Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Asia','region2']='Asia'
region_pc_totals.loc[region_pc_totals.region=='South Asia','region2']='Asia'
region_pc_totals.loc[region_pc_totals.region=='East Asia and Pacific','region2']='Asia'

In [163]:
region_pc_totals.columns

Index(['region', 'sector', 'scenario', 'variable', 'capital vs maintenance',
       'data', 'ssp', 'year', 'GDP', 'cost_bUSD', 'discount_factor',
       'cost_bUSD_disc', 'region2'],
      dtype='object')

In [164]:
region_pc_totals.sample(5)

,region,sector,scenario,variable,capital vs maintenance,data,ssp,year,GDP,cost_bUSD,discount_factor,cost_bUSD_disc,region2
1402,Eastern Europe and Central Asia,coastal protection,min,% GDP,maintenance,0.0020,ssp2,2025,4.702948e+12,0.094059,0.591898,0.055673,Eastern Europe and Central Asia
1182,Eastern Europe and Central Asia,energy,best,% GDP,maintenance,1.1000,ssp2,2029,5.423777e+12,59.661545,0.468839,27.971660,Eastern Europe and Central Asia
3260,South Asia,river protection,min,% GDP,maintenance,0.0096,ssp3,2027,7.101803e+12,0.681773,0.526788,0.359150,Asia
283,all_countries,wash,min,% GDP,capital,0.3200,ssp2,2026,5.717993e+13,182.975791,0.558395,102.172726,all_countries
2443,South Asia,river protection,best,% GDP,capital,0.3700,ssp2,2026,7.090173e+12,26.233639,0.558395,14.648727,Asia


In [165]:
region_pc_totals = region_pc_totals.set_index(['sector','scenario','capital vs maintenance','region2','year'])[['cost_bUSD','cost_bUSD_disc','GDP']].sum(level=['scenario','sector','capital vs maintenance','region2','year']).drop("all_countries",level="region2")

In [166]:
region_pc_totals.sample(10)

cost_bUSD  \
scenario sector             capital vs maintenance region2                         year               
best     coastal protection maintenance            Eastern Europe and Central Asia 2018    0.365809   
                            capital                Asia                            2030   55.362546   
min      wash               capital                Eastern Europe and Central Asia 2028   17.303780   
         transport          maintenance            Africa                          2022   56.483393   
best     wash               maintenance            Eastern Europe and Central Asia 2022    5.529746   
         river protection   maintenance            Asia                            2018    1.893171   
max      energy             maintenance            Asia                            2030  291.148239   
min      irrigation         capital                Africa                          2023   14.904347   
         coastal protection maintenance            Eastern Europe and Central Asia 2020    0.079082   
max      transport          maintenance            Asia                            2021  639.935417   

                                                                                         cost_bUSD_disc  \
scenario sector             capital vs maintenance region2                         year                   
best     coastal protection maintenance            Eastern Europe and Central Asia 2018        0.325569   
                            capital                Asia                            2030       24.486907   
min      wash               capital                Eastern Europe and Central Asia 2028        8.599448   
         transport          maintenance            Africa                          2022       39.818564   
best     wash               maintenance            Eastern Europe and Central Asia 2022        3.898252   
         river protection   maintenance            Asia                            2018        1.684915   
max      energy             maintenance            Asia                            2030      128.775147   
min      irrigation         capital                Africa                          2023        9.912242   
         coastal protection maintenance            Eastern Europe and Central Asia 2020        0.062641   
max      transport          maintenance            Asia                            2021      478.196970   

                                                                                                  GDP  
scenario sector             capital vs maintenance region2                         year                
best     coastal protection maintenance            Eastern Europe and Central Asia 2018  3.658094e+12  
                            capital                Asia                            2030  3.988332e+13  
min      wash               capital                Eastern Europe and Central Asia 2028  5.243570e+12  
         transport          maintenance            Africa                          2022  6.645105e+12  
best     wash               maintenance            Eastern Europe and Central Asia 2022  4.253650e+12  
         river protection   maintenance            Asia                            2018  2.100782e+13  
max      energy             maintenance            Asia                            2030  3.988332e+13  
min      irrigation         capital                Africa                          2023  7.016315e+12  
         coastal protection maintenance            Eastern Europe and Central Asia 2020  3.954118e+12  
max      transport          maintenance            Asia                            2021  2.509551e+13

In [167]:
region_pc_totals2 = region_pc_totals[['cost_bUSD','cost_bUSD_disc']].sum(level=['scenario','capital vs maintenance','region2','year']).copy()

In [168]:
region_pc_totals2['GDP'] = region_pc_totals.GDP.mean(level=['scenario','capital vs maintenance','region2','year'])

In [169]:
region_pc_totals2.sample(3)

cost_bUSD  \
scenario capital vs maintenance region2                         year              
min      maintenance            Africa                          2023  88.116779   
                                                                2017  65.413967   
         capital                Eastern Europe and Central Asia 2028  57.875088   

                                                                      cost_bUSD_disc  \
scenario capital vs maintenance region2                         year                   
min      maintenance            Africa                          2023       58.602691   
                                                                2017       61.711290   
         capital                Eastern Europe and Central Asia 2028       28.762146   

                                                                               GDP  
scenario capital vs maintenance region2                         year                
min      maintenance            Africa                          2023  6.909857e+12  
                                                                2017  5.194217e+12  
         capital                Eastern Europe and Central Asia 2028  5.283623e+12

In [170]:
region_pc_totals2['new_data'] = 100*1e9*region_pc_totals2.cost_bUSD/region_pc_totals2.GDP

In [171]:
region_pc_totals_mean = region_pc_totals2['new_data'].mean(level=['scenario','capital vs maintenance','region2']).to_frame()

In [172]:
region_pc_totals_mean['cost_bUSD_disc'] = 1/15*region_pc_totals2['cost_bUSD_disc'].sum(level=['scenario','capital vs maintenance','region2'])

In [173]:
region_pc_totals_mean.unstack("capital vs maintenance")

new_data              \
capital vs maintenance                      capital maintenance   
scenario region2                                                  
best     Africa                            6.404040    1.758783   
         Asia                              4.044901    2.538012   
         Eastern Europe and Central Asia   5.797000    3.042000   
         Latin America and Caribbean       3.340000    1.036000   
max      Africa                           12.517166    2.237943   
         Asia                              7.004895    3.417769   
         Eastern Europe and Central Asia   5.166284    3.120898   
         Latin America and Caribbean       7.568284    1.380169   
min      Africa                            2.939233    1.277596   
         Asia                              1.589688    2.544817   
         Eastern Europe and Central Asia   1.098990    2.394260   
         Latin America and Caribbean       1.869217    0.789886   

                                         cost_bUSD_disc              
capital vs maintenance                          capital maintenance  
scenario region2                                                     
best     Africa                              315.635356   86.687476  
         Asia                                792.353132  497.174612  
         Eastern Europe and Central Asia     182.873965   95.963878  
         Latin America and Caribbean         254.675000   78.995000  
max      Africa                              618.981138  110.762767  
         Asia                               1383.861351  676.346066  
         Eastern Europe and Central Asia     163.575944   98.938796  
         Latin America and Caribbean         578.832406  105.617416  
min      Africa                              145.402183   63.000495  
         Asia                                314.519356  500.092781  
         Eastern Europe and Central Asia      34.776291   75.464633  
         Latin America and Caribbean         142.849285   60.207072

In [131]:
region_pc_totals_mean.unstack("capital vs maintenance").to_excel("C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\aggregated_regions_numbers.xlsx")

In [174]:
region_pc3 = region_pc_totals.copy()

In [175]:
region_pc3['new_data'] = 100*1e9*region_pc3.cost_bUSD/region_pc3.GDP

In [177]:
region_pc3_mean = region_pc3['new_data'].mean(level=['sector','scenario','capital vs maintenance','region2']).to_frame()

In [178]:
region_pc3_mean['cost_bUSD_disc'] = 1/15*region_pc3['cost_bUSD_disc'].sum(level=['sector','scenario','capital vs maintenance','region2'])

In [192]:
region_pc4 = region_pc3_mean.stack().reset_index().rename(columns={"level_4":"usd vs percent",0:"data"})

In [193]:
region_pc4 = region_pc4.set_index(['region2','sector','scenario','capital vs maintenance',"usd vs percent"]).data.unstack("sector")

In [194]:
region_pc4['flood protection'] = region_pc4['coastal protection']+region_pc4['river protection']

In [195]:
region_pc5 = region_pc4.drop(['coastal protection','river protection'],axis=1).stack().to_frame().reset_index().rename(columns={0:"data"})

In [197]:
to_save = region_pc5.reset_index().sort_values(['region2','sector','scenario','capital vs maintenance','usd vs percent'])

In [198]:
sector_dic = {'energy':'Energy','irrigation':'Irrigation','wash':"Water supply and sanitation","transport":"Transport","flood protection":"Flood protection"}
scenario_dic = {'best':'Preferred','min':'Low spending','max':'High spending'}
var_dic = {"new_data":"Percent of GDP","cost_bUSD_disc":"Billion USD"}

In [199]:
to_save['sector'] = to_save.sector.replace(sector_dic)
to_save['scenario'] = to_save.scenario.replace(scenario_dic)
to_save['usd vs percent'] = to_save['usd vs percent'].replace(var_dic)

In [201]:
to_save.to_csv("C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_numbers_large_regions.csv")